# Music Perception and Cognition - Audio Frequency Discrimination Test

This notebook generates audio stimuli for testing frequency discrimination in three contexts:
1. **Task 1**: Isolated tone pairs with varying frequency differences
2. **Task 2**: Melodic context (Happy Birthday) with detuned notes
3. **Task 3**: Unfamiliar melodic context with detuned notes

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eacevedo1/mpc-questionary/blob/main/stimuli_generation.ipynb)

# Import Libraries

Import necessary Python libraries for audio generation and playback.

In [ ]:
import numpy as np
from IPython.display import Audio, display

In [ ]:
import os
from scipy.io import wavfile

# Create directories for audio output
os.makedirs("data/task1_isolated_tones", exist_ok=True)
os.makedirs("data/task2_melody_D4", exist_ok=True)
os.makedirs("data/task2_melody_F4", exist_ok=True)
os.makedirs("data/task3_melody_D4", exist_ok=True)
os.makedirs("data/task3_melody_F4", exist_ok=True)

print("Audio output directories created:")
print("  - data/task1_isolated_tones/")
print("  - data/task2_melody_D4/")
print("  - data/task2_melody_F4/")
print("  - data/task3_melody_D4/")
print("  - data/task3_melody_F4/")

# Constants

Define global constants for audio generation:
- **FS**: Sampling frequency (44.1 kHz)
- **HAPPY_DURATIONS**: Note durations for the Happy Birthday melody
- **NOTE_SILENCE**: Silence duration between notes
- **freq_map**: Mapping of note names to their frequencies in Hz

In [ ]:
# ------------------------------
# GLOBALS
# ------------------------------
FS = 44100
HAPPY_DURATIONS = [0.3, 0.2, 0.6, 0.6, 0.6, 1.0]
NOTE_SILENCE = 0.05

# Base frequencies for "note names" (used only as labels + base Hz)
freq_map = {
    "C-4": 261.63, "D-4": 293.66, "E-4": 329.63,
    "F-4": 349.23, "G-4": 392.00, "A-3": 220.00, "B-3": 246.94,
}

# Task 1 - Isolated Tone Pairs

This task generates pairs of pure tones to test frequency discrimination in isolation.

**Procedure:**
- Reference tone: D-4 (293.66 Hz)
- Comparison tones: Reference ± Δf, where Δf ranges from -10 Hz to +10 Hz
- Each pair consists of: reference tone → silence (0.6s) → comparison tone
- Duration of each tone: 0.6 seconds

**Purpose:** Measure the Just Noticeable Difference (JND) for pitch in isolated tones.

In [ ]:
COMPARISON_SILENCE = 0.6 # duration between tones (s)
DUR = 0.6   # duration of each tone (s)
NOTE = freq_map["D-4"]  # reference frequency in Hz
AMPLITUDE = 0.7  # amplitude of tones (0.0 to 1.0)

def make_tone(freq, dur=DUR, fs=FS):
    t = np.linspace(0, dur, int(fs * dur), endpoint=False)
    tone = AMPLITUDE * np.sin(2 * np.pi * freq * t)
    
    # short fade in/out to avoid clicks
    f = int(0.01 * fs)
    if f > 0:
        fade_in = np.linspace(0, 1, f)
        fade_out = fade_in[::-1]
        tone[:f] *= fade_in
        tone[-f:] *= fade_out

    return tone.astype(np.float32)

silence = np.zeros(int(FS * COMPARISON_SILENCE), dtype=np.float32)

for df in range(-10, 11):  # -10 Hz to +10 Hz
    ref = make_tone(NOTE)
    comp = make_tone(NOTE + df)
    pair = np.concatenate([ref, silence, comp])

    # Save audio file
    filename = f"data/task1_isolated_tones/tone_pair_delta_{df:+03d}Hz.wav"
    wavfile.write(filename, FS, pair)

    print(f"Δf = {df:+d} Hz (comparison = {NOTE + df:.1f} Hz) - Saved: {filename}")
    display(Audio(pair, rate=FS))

# Task 2 - Happy Birthday Melody Context

This task tests frequency discrimination within a melodic context using the first phrase of "Happy Birthday".

**Melody:** C-4, C-4, D-4, C-4, F-4, E-4

The experiment is conducted in two conditions:
1. **Detuning D-4**: The third note is detuned by ±10 Hz
2. **Detuning F-4**: The fifth note is detuned by ±10 Hz

**Procedure:**
- Reference melody: original melody with no detuning
- Comparison melodies: one note detuned by Δf (range: -10 Hz to +10 Hz)
- Each pair consists of: reference melody → silence (1.0s) → comparison melody

**Purpose:** Investigate whether melodic context affects pitch discrimination ability compared to isolated tones.

## Task 2.1 - Detuning D-4 (Third Note)

Testing pitch discrimination when the third note (D-4) in the melody is detuned.

In [ ]:
import numpy as np
from IPython.display import Audio, display

FS = 44100
HAPPY_DURATIONS = [0.3, 0.2, 0.6, 0.6, 0.6, 1.0]
NOTE_SILENCE = 0.05
COMPARISON_SILENCE = 1.0
AMPLITUDE = 0.7

# Base frequencies for "note names" (used only as labels + base Hz)
freq_map = {
    "C-4": 261.63, "D-4": 293.66, "E-4": 329.63,
    "F-4": 349.23, "G-4": 392.00, "A-3": 220.00, "B-3": 246.94,
}

melody = ["C-4", "C-4", "D-4", "C-4", "F-4", "E-4"]
note_to_modify = "D-4"

def tone(freq, dur):
    # Shorten note so fade-out does not overlap silence
    FADE_TIME = 0.15 * dur
    dur = max(0, dur - FADE_TIME)

    # Generate tone
    t = np.linspace(0, dur, int(FS * dur), endpoint=False)
    w = AMPLITUDE * np.sin(2 * np.pi * freq * t).astype(np.float32)
    fN = int(0.01 * FS)
    w[:fN] *= np.linspace(0, 1, fN)
    w[-fN:] *= np.linspace(1, 0, fN)
    return w

sil = np.zeros(int(FS * NOTE_SILENCE), np.float32)
gap = np.zeros(int(FS * COMPARISON_SILENCE), np.float32)

def build(delta):
    parts = []
    for (n, dur) in zip(melody, HAPPY_DURATIONS):
        f = freq_map[n]
        if n == note_to_modify:
            f += delta
        parts.append(tone(f, dur))
        parts.append(sil)
    return np.concatenate(parts)

ref = build(0)

for df in range(-10, 11):
    comp = build(df)
    pair = np.concatenate([ref, gap, comp])

    # Save audio file
    filename = f"data/task2_melody_D4/melody_D4_delta_{df:+03d}Hz.wav"
    wavfile.write(filename, FS, pair)

    print(f"Δf on {note_to_modify} = {df:+d} Hz - Saved: {filename}")
    display(Audio(pair, rate=FS))

## Task 2.2 - Detuning F-4 (Fifth Note)

Testing pitch discrimination when the fifth note (F-4) in the melody is detuned.

In [ ]:
import numpy as np
from IPython.display import Audio, display

FS = 44100
HAPPY_DURATIONS = [0.3, 0.2, 0.6, 0.6, 0.6, 1.0]
NOTE_SILENCE = 0.05
COMPARISON_SILENCE = 1.0
AMPLITUDE = 0.7

# Base frequencies for "note names" (used only as labels + base Hz)
freq_map = {
    "C-4": 261.63, "D-4": 293.66, "E-4": 329.63,
    "F-4": 349.23, "G-4": 392.00, "A-3": 220.00, "B-3": 246.94,
}

melody = ["C-4", "C-4", "D-4", "C-4", "F-4", "E-4"]
note_to_modify = "F-4"

def tone(freq, dur):
    # Shorten note so fade-out does not overlap silence
    FADE_TIME = 0.15 * dur
    dur = max(0, dur - FADE_TIME)

    # Generate tone
    t = np.linspace(0, dur, int(FS * dur), endpoint=False)
    w = AMPLITUDE * np.sin(2 * np.pi * freq * t).astype(np.float32)
    fN = int(0.01 * FS)
    w[:fN] *= np.linspace(0, 1, fN)
    w[-fN:] *= np.linspace(1, 0, fN)
    return w

sil = np.zeros(int(FS * NOTE_SILENCE), np.float32)
gap = np.zeros(int(FS * COMPARISON_SILENCE), np.float32)

def build(delta):
    parts = []
    for (n, dur) in zip(melody, HAPPY_DURATIONS):
        f = freq_map[n]
        if n == note_to_modify:
            f += delta
        parts.append(tone(f, dur))
        parts.append(sil)
    return np.concatenate(parts)

ref = build(0)

for df in range(-10, 11):
    comp = build(df)
    pair = np.concatenate([ref, gap, comp])

    # Save audio file
    filename = f"data/task2_melody_F4/melody_F4_delta_{df:+03d}Hz.wav"
    wavfile.write(filename, FS, pair)

    print(f"Δf on {note_to_modify} = {df:+d} Hz - Saved: {filename}")
    display(Audio(pair, rate=FS))

# Task 3 - Unfamiliar Melody Context

This task tests frequency discrimination within an unfamiliar melodic context.

**Melody:** E-4, C-4, F-4, D-4, C-4, E-4

The experiment is conducted in two conditions:
1. **Detuning D-4**: The fourth note is detuned by ±10 Hz
2. **Detuning F-4**: The third note is detuned by ±10 Hz

**Procedure:**
- Reference melody: original melody with no detuning
- Comparison melodies: one note detuned by Δf (range: -10 Hz to +10 Hz)
- Each pair consists of: reference melody → silence (1.0s) → comparison melody

**Purpose:** Compare pitch discrimination ability between familiar (Task 2) and unfamiliar melodic contexts.

In [ ]:
FS = 44100
NOTE_SILENCE = 0.05
COMPARISON_SILENCE = 1.0
AMPLITUDE = 0.7

# Task 3 melody and durations (unfamiliar melody)
unfamiliar_melody = ["E-4", "C-4", "F-4", "D-4", "C-4", "E-4"]
unfamiliar_durations = [0.4, 0.5, 0.6, 0.4, 0.5, 0.8]

# Base frequencies for "note names" (used only as labels + base Hz)
freq_map = {
    "C-4": 261.63, "D-4": 293.66, "E-4": 329.63,
    "F-4": 349.23, "G-4": 392.00, "A-3": 220.00, "B-3": 246.94,
}

def tone(freq, dur):
    # Shorten note so fade-out does not overlap silence
    FADE_TIME = 0.15 * dur
    dur = max(0, dur - FADE_TIME)

    # Generate tone
    t = np.linspace(0, dur, int(FS * dur), endpoint=False)
    w = AMPLITUDE * np.sin(2 * np.pi * freq * t).astype(np.float32)
    fN = int(0.01 * FS)
    w[:fN] *= np.linspace(0, 1, fN)
    w[-fN:] *= np.linspace(1, 0, fN)
    return w

sil = np.zeros(int(FS * NOTE_SILENCE), np.float32)
gap = np.zeros(int(FS * COMPARISON_SILENCE), np.float32)

def build(delta, note_to_modify):
    parts = []
    for (n, dur) in zip(unfamiliar_melody, unfamiliar_durations):
        f = freq_map[n]
        if n == note_to_modify:
            f += delta
        parts.append(tone(f, dur))
        parts.append(sil)
    return np.concatenate(parts)

for note_to_modify in ["D-4", "F-4"]:
    out_dir = f"data/task3_melody_{note_to_modify.replace('-', '')}"
    os.makedirs(out_dir, exist_ok=True)

    ref = build(0, note_to_modify)

    for df in range(-10, 11):
        comp = build(df, note_to_modify)
        pair = np.concatenate([ref, gap, comp])

        filename = f"{out_dir}/melody_{note_to_modify.replace('-', '')}_delta_{df:+03d}Hz.wav"
        wavfile.write(filename, FS, pair)

        print(f"Δf on {note_to_modify} = {df:+d} Hz - Saved: {filename}")
        display(Audio(pair, rate=FS))

# Download Stimuli Zip File

In [ ]:
import zipfile
import os
from google.colab import files

# Define the directory to zip and the output zip file name
directory_to_zip = 'data'
output_zip_filename = 'audio_data.zip'

# Create a ZipFile object
with zipfile.ZipFile(output_zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files_in_dir in os.walk(directory_to_zip):
        for file in files_in_dir:
            file_path = os.path.join(root, file)
            # Add file to zip, preserving directory structure relative to directory_to_zip
            zipf.write(file_path, os.path.relpath(file_path, directory_to_zip))

print(f"'{output_zip_filename}' created successfully.")

# Download the zip file
files.download(output_zip_filename)